# Visuliazation of some paramaters of robot
- this jupyter notbook show the variables related to the robot


## import

In [1]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import jupyros as jr
import rospy
from nav_msgs.msg import Odometry
from matplotlib import animation, rc
import rt2_assignment1.msg
from geometry_msgs.msg import Twist
from std_msgs.msg import Float32
from ipywidgets import Button, Layout, ButtonStyle, GridBox, VBox, HBox 
import ipywidgets as widgets
from rt2_assignment1.srv import Command
from IPython.display import display
from tf import transformations
import actionlib
import actionlib.msg
import collections
import math

rospy.init_node('visualized_node')

## display the movemennt of the robot

- an xy graph showing the robot’s position and the orientation 

In [ ]:
# First set up the figure and the axis 
fig, ax = plt.subplots()

ax.set_xlim(( -10, 10))
ax.set_ylim((-10, 10))
plt.xlabel("X")
plt.ylabel("Y")
plt.title("Robot positon")

line, = ax.plot([], [], 'ro')
x_pos=0
y_pos=0
x_data=collections.deque(maxlen=300)
y_data=collections.deque(maxlen=300)
yaw=0
theta=collections.deque(maxlen=300)
arrow = ax.quiver(1,1,1,1, alpha = 0)


def init():
    line.set_data([], [])
    return (line,)


def odom_callback(msg):
    global x_data,y_data,yaw,theta,actual_linear_vel,actual_angular_vel
    x_pos=msg.pose.pose.position.x
    print(x_pos)
    y_pos=msg.pose.pose.position.y
    y_data.append(y_pos)
    x_data.append(x_pos)

    quaternion = (
        msg.pose.pose.orientation.x,
        msg.pose.pose.orientation.y,
        msg.pose.pose.orientation.z,
        msg.pose.pose.orientation.w)
    euler = transformations.euler_from_quaternion(quaternion)
    yaw = euler[2]
    theta.append(yaw)
    
def animate(i):
    global x_data,y_data,yaw,arrow,theta
    global arrow
    arrow.remove()
    line.set_data(x_data, y_data)
    
#     point = {
#         'start': [x_data[-1], y_data[-1]],
#         'end': [x_data[-1]+x_data[-1]*math.cos(yaw), y_data[-1]+y_data[-1]*math.sin(yaw)]
#     }
#     arrow = plt.Arrow(x=x_pos,y=y_pos,dx=10*np.cos(yaw),dy=10*np.cos(yaw),width=0.01,head_width=0.05,head_length=0.2,length_includes_head=True,color='k')
#     arrow=ax.annotate('', xy=point['end'], xytext=point['start'],
#                 arrowprops=dict(shrink=0, width=1, headwidth=8, 
#                                 headlength=10, connectionstyle='arc3',
#                                 facecolor='gray', edgecolor='gray')
#                )
#     ax.add_patch(arrow)
    delta=0.2
    x = x_data[-1]+delta*np.cos(theta[-1])
    y = y_data[-1]+delta*np.sin(theta[-1])
    a = np.cos(theta[-1])++delta*np.cos(theta[-1]) 
    b = np.sin(theta[-1])+delta*np.sin(theta[-1])
    arrow = ax.quiver(x, y, a, b,scale = 0.5, color='k', pivot = 'tail', units='xy', alpha=0.6, edgecolor = 'black')
    
    return (line,arrow)



jr.subscribe('/odom', Odometry, odom_callback)

# call the animator. blit=True means only re-draw the parts that have changed.

anim = animation.FuncAnimation(fig, animate, init_func=init, blit=True)

## compare the actual velocity with cmd_vel

- a line plot for visualizing cmd_vel vs. actual velocity (for linear and angular velocity)

In [2]:
fig1, ax1 = plt.subplots()

ax1.set_xlim(( 0, 100))
ax1.set_ylim((-1.2, 1.2))

linear_cmd_vel=[]
angular_cmd_vel=[]
actual_linear_vel=[]
actual_angular_vel=[]

time=[]
odom_time=[]

line1, = ax1.plot([], [], 'r-')
line2, = ax1.plot([], [], 'b-')
line3, = ax1.plot([], [], 'c-')
line4, = ax1.plot([], [], 'y-')
zero_time = rospy.get_time()

def init1():
    line1.set_data([], [])
    line2.set_data([], [])
    line3.set_data([], [])
    line4.set_data([], [])
    return (line1,line2,line3,line4)

def twist_callback(msg):
    global linear_cmd_vel,angular_cmd_vel,zero_time,time
    linear_cmd_vel.append(msg.linear.x)
    angular_cmd_vel.append(msg.angular.z)
    time.append(rospy.get_time()-zero_time)

def odom_vel_callback(msg):
    global actual_linear_vel,actual_angular_vel,odom_time,zero_time
    actual_linear_vel.append(msg.twist.twist.linear.x)
    actual_angular_vel.append(-1.85*msg.twist.twist.angular.z)
    odom_time.append(rospy.get_time()-zero_time)

def animate1(i):
    global linear_cmd_vel,angular_cmd_vel,time,odom_time,actual_linear_vel,actual_angular_vel
    line1.set_data(time,linear_cmd_vel)
    line2.set_data(time, angular_cmd_vel)
    line3.set_data(odom_time, actual_linear_vel)
    line4.set_data(odom_time, actual_angular_vel)

    return (line1,line2,line3,line4)

jr.subscribe('/odom', Odometry, odom_vel_callback)
jr.subscribe('/cmd_vel', Twist, twist_callback)

anim = animation.FuncAnimation(fig1, animate1, init_func=init1,
                               frames=100, interval=20, blit=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
print(linear_cmd_vel)


In [ ]:
print(actual_linear_vel)

In [ ]:
print(actual_angular_vel)

- a boxplot displaying the number of reached targets and cancelled targets


## display the robot status
- a boxplot displaying the number of reached targets and cancelled targets
- subscribe /go_to_point/result and update the number of goal and cancel

In [3]:
ngoal=0
ncancel=0
fig2, ax2 = plt.subplots()

ax2.set_xlim(( 0, 2))
ax2.set_ylim((0.0, 10))

def result_callback(msg):
    global ngoal, ncancel
    if msg.result.reached==True:
        ngoal=ngoal+1
        print(ngoal)
    #elif msg.result.reached==False:
    else:
        ncancel=ncancel+1

def animate2(i):
    global ngoal,ncancel
    index=1
    ax2.bar(index,ngoal,label="the number of goal", align="center", color='blue', width=0.35)
    ax2.bar(index+0.35, ncancel, align='center', width=0.35,color='r', label='the number of cancel') 
        
jr.subscribe('/go_to_point/result', rt2_assignment1.msg.MotionActionResult, result_callback)

anim = animation.FuncAnimation(fig2, animate2, 
                               frames=100, interval=20,blit=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [4]:
print(ngoal)

11


## show the required time to reach goals
- a hist plot showing the time required to reach targets
- prepare the topic /reached_time

In [5]:
fig3, ax3 = plt.subplots()
ax3.set_ylim((0.0, 5))
goal_time_array=[]
index=0
def init3():
    print('init')
def reached_callback(msg):
    global goal_time_array
    goal_time_array.append(msg.data)
    print(goal_time_array)
    
def animate3(i):
    global goal_time_array
    ax3.hist(goal_time_array,bins=10, align='mid',ec="yellow", fc="green", alpha=0.5) 


jr.subscribe('/reached_time', Float32 , reached_callback)

anim = animation.FuncAnimation(fig3, animate3,init_func=init3, 
                               frames=100, interval=200,blit=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …